# Langchain and Anthropic

Claude is chat-based model, meaning it is trained on conversation data. However, it is a text based API, meaning it takes in single string. It expects this string to be in a particular format. This means that it is up the user to ensure that is the case. LangChain provides several utilities and helper functions to make sure prompts that you write - whether formatted as a string or as a list of messages - end up formatted correctly.

Because Claude is chat-based but accepts a string as input, it can be treated as either a LangChain `ChatModel` or `LLM`. This means there are two wrappers in LangChain - ChatAnthropic and Anthropic. It is generally recommended to use the ChatAnthropic wrapper, and format your prompts as ChatMessages (we will show examples of this below). This is because it keeps your prompt in a general format that you can easily then also use with other models (should you want to). However, if you want more fine-grained control over the prompt, you can use the Anthropic wrapper - we will show and example of this as well. The `Anthropic` wrapper however is **deprecated**, as all functionality can be achieved in a more generic way using `ChatAnthropic`.

Ref: <https://python.langchain.com/docs/integrations/platforms/anthropic>

## Best Practices

#### No System Messages

Anthropic models are not trained on the concept of a "system message". We have worked with the Anthropic team to handle them somewhat appropriately (a Human message with an admin tag) but this is largely a hack and it is recommended that you do not use system messages.

#### AI Messages Can Continue

A completion from Claude is a continuation of the last text in the string which allows you further control over Claude's output.

For example, putting words in Claude's mouth in a prompt like this:

>`\n\nHuman: Tell me a joke about bears\n\nAssistant: What do you call a bear with no teeth?`

This will return a completion like this `A gummy bear!` instead of a whole new assistant message with a different random bear joke.

In [3]:
# !pip install -Uq boto3 anthropic-bedrock langchain rich

In [4]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/utils")

In [5]:
import re
import warnings
from pathlib import Path
from types import FunctionType

import boto3
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT
from IPython.display import Markdown
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chat_models import ChatAnthropic
from langchain.llms.bedrock import Bedrock
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import HumanMessage
from rich import print
from utils import get_inference_parameters

warnings.filterwarnings("ignore")

%load_ext rich
%load_ext autoreload
%autoreload 2

The rich extension is already loaded. To reload it, use:
  %reload_ext rich
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup

### Initialize Bedrock LLM

Here we utilize `langchain.llms.bedrock.Bedrock` class to initialize our LLM.

In [6]:
region = "us-west-2"

# get bedrock runtime client
client = boto3.client("bedrock-runtime", region_name=region)

bedrock_model_id = "anthropic.claude-v2"  # Bedrock model_id
model_kwargs = get_inference_parameters("anthropic")  # Model kwargs for Anthropic LLMs
# print(model_kwargs)

bedrock_model = Bedrock(
    client=client,
    model_id=bedrock_model_id,
    model_kwargs=model_kwargs,
    region_name=region,
)  # Initalize LLM

#### Creating Anthropic prompts with ChatPromptTemplate

When working with ChatModels, it is preferred that you design your prompts as `ChatPromptTemplates`.

Anthropic expects chat prompts to be formatted in a specific manner.

```text
\n\nHuman: Please translate the below text to french.\n\nAssistant:
```

We can chat messages by using the `human` and `assistant` prefixes when initializing the prompt. LangChain automatically adds the prefix and suffix.

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        ("human", "Hi! How are you?"),
        ("assistant", "I'm doing well. Thanks for asking."),
        ("human", "Tell me a joke about {topic}"),
    ]
)
print(prompt)

ChatPromptTemplate(
    input_variables=['topic'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')
        ),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hi! How are you?')),
        AIMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], template="I'm doing well. Thanks for asking.")
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['topic'], template='Tell me a joke about {topic}')
        )
    ]
)

You can then invoke the chain like below:

*NOTE:* The below example is using Langchain Expression Language (LCEL)

Refer to <https://python.langchain.com/docs/expression_language> for more details.

In [8]:
chain = prompt | bedrock_model
chain.invoke({"topic": "cats"})

" Here's a silly joke about cats:\n\nWhy don't cats play poker in the jungle? Too many cheetahs!"

### Creating Prompts using `HumanMessage`

Below we use `HumanMessage` from `langchain.schema` to construct `messages` for the chat model.

We then create a `ChatPromptTemplate` from these messages to invoke this chain.


*NOTE: As there are no input variables to be passed, we send an empty `input` string*

In [9]:
messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
prompt = ChatPromptTemplate.from_messages(messages=messages)

chain = prompt | bedrock_model
output = chain.invoke({"input": ""})

Markdown(output)

 Voici la traduction en français : J'adore programmer.

##### Another Variation

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "You're an AI assistant who's good at {ability}. Please answer this {question}",
        )
    ]
)
chain = prompt | bedrock_model

output = chain.invoke(
    {"ability": "computerscience", "question": "What are sorting algorithms"}
)

Markdown(output)

 Sorting algorithms are algorithms that put elements of a list in a certain order. Some common sorting algorithms include:

- Bubble sort - Repeatedly steps through a list, compares adjacent elements, and swaps them if they are in the wrong order.

- Insertion sort - Builds up a sorted list by inserting each new element into the correct position in the already sorted portion of the list.

- Selection sort - Finds the smallest remaining element and adds it to the end of the sorted list.

- Merge sort - Recursively divides a list into smaller sublists, sorts each sublist, then merges the sublists back together in sorted order. Uses a divide-and-conquer approach.

- Quicksort - Chooses a pivot element and partitions the list into two sublists - elements less than the pivot and elements greater than the pivot. Recursively sorts the sublists.

- Heapsort - Uses a binary heap data structure to sort the list. Builds a heap from the list, removes the largest element from the heap and places it at the end of the list, then reconstructs the heap with the remaining elements.

The choice of sorting algorithm depends on things like efficiency, memory usage, stability, etc. Some algorithms like merge sort and heapsort are faster but require more memory. Others like insertion sort are simple to implement but slower. The optimal algorithm depends on the specific use case.

#### Prompts with PromptTemplate

We can see that under the hood LangChain is not appending any prefix/suffix to SystemMessage's. This is because `Anthropic` has no concept of `SystemMessage`. 

Anthropic requires all prompts to end with assistant messages. This means if the last message is not an assistant message, the suffix `Assistant:` will automatically be inserted.

If you decide instead to use a normal `PromptTemplate` (one that just works on a single string) then we need do the following:

- Prefix our prompt string with `HUMAN_PROMPT` and suffix with `AI_PROMPT`

In [11]:
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
# Format prompt for Anthropic
anthropic_prompt = PromptTemplate.from_template(
    f"{HUMAN_PROMPT}{prompt.template}{AI_PROMPT}"
)
print(anthropic_prompt)

# Invoke the Chain
chain = anthropic_prompt | bedrock_model
chain.invoke({"topic": "otters"})

PromptTemplate(input_variables=['topic'], template='\n\nHuman:Tell me a joke about {topic}\n\nAssistant:')

" Here's a silly joke about otters:\n\nWhat do you call an otter with a cold? A snotter!"